Need to be aware of spaces when defining variables, recall spaces are used to separate options/flags, so doing `foo=bar` will work, whereas `foo = bar` will return an error (it is interpreted as calling the `foo` program, with `=` as the first argument and `bar` as the second argument)

We can put functions into files, execute and load it into the shell by `source`

file `mcd.sh`:
```
mcd() {
    mkdir "$1"
    cd "$1"
   }
```
sourcing it in shell:
`source mcd.sh`

`$_`: last argument of the previous command, this can be handy at times, for example creating a directory and changing directory to it:
```
mkdir test
cd $_
```

`$@`: expands to all arguments, handy when we don't know how many arguments there will be beforehand, example: `for file in "$@"; do`

`$#`: number of arguments

`$$`: process id

`$?`: error message of the previous command, if `echo $?` returned 0, it meant the previous command executed successfully

`$0`: name of the script

`$1`~`$9`: first to the 9th command

Another command that might come in handy is `!!`, for example, if we were to do something but was denied permission, then we can do:

```mkdir /directory/with/no/permission
sudo !!```

The `!!` will be replaced with the previous command

Getting the output of a command into a variable:

In [1]:
!foo=$(pwd)
!echo "we are in $(pwd)"

we are in /home/lukezhu/Code/missing-semester


### Process substitution:
`<(some_command)` makes the input or output of a command to appear as a file, this can come in handy for commands that doesn't take input from stdin but from files, for example, the following command concatenates the file names in the current and parent directory and prints it out on the terminal:

In [10]:
%%bash
cat <(ls ..) <(ls)

fine-scale-mutation-spectrum
missing-semester
mushi
mushi_lz
mut_R
mutyper
Lecture 1.ipynb
Lecture 2.ipynb
missing


Example used in class:
```
for file in "$@"; do
    grep foobar "$file" > dev/null 2> /dev/null 
    if [[ "$?" -ne 0 ]]; then 
        echo "File $file does not have any foobar
    fi
done
```
Note: the line `grep foobar "$file" > dev/null 2> /dev/null` meant we are not interested in the result from stdout and stderr, so we dump it to /dev/null, 2> is for stderr

### Globbing:
wildcard matching with `*`, we can also use `$` for a one character wildcard matching, for example, if we have directories called `project1`, `project2` and `project42`, and we only want the projects with single digits, then we do `project$`

We can also use `{}`, for example, if we want to create files `foo1, foo2, foo10`, instead of typing all file names 1 by 1, we can just do `touch foo{1,2,10}`. If we do something like `touch project{1,2}/testP{1,2,3}.py`, then the shell will expand everything inside the curly braces into its cartesian product. We can also do things like `touch {foo, bar}/{a...j}`, in which case, the shell will create files `a` to `j` under the directory `foo` and `bar`

### Shebang
In the exercise from lecture 1, we already knew that #! will specify the interpreter directory, so that's why in Python scripts, we commonly see the first line as something similar to `#!usr/local/bin/python`, but what if we don't know where the `python` interpreter is, then we can try 

`#!usr/bin/env python`

instead, in this case, we make use of the `env` variable, and attempt to search for `python` in the `env`

### Shellcheck
Shellcheck helps check your file, finds if there are any errors and gives warning, usage:

`shellcheck some_file`

### tldr
Last lecture, we learned about using `man` to access the manual, but sometimes the manual is very long, and instead there is this other program called `tldr` that we can download, and it gives more succint instructions on how to run a program, usage:

`tldr some_command`

### How to find files
`find`, example: 

`find . -name src -type d`, `.` means the current directory, `d` means we want to find directories

`find . -path **/yest/*.py`, `**` means some number of folders

`find . -mtime -1`, `-mtime` stands for the the time the file was modified (in the unit of days)

We can also execute commands after we find the files:

`find . -name *.tmp -exec rm {} \;`

### More on grep
We can search recursively through a directory with `grep` using the `-R` flag, a potential use case for this is when we have multiple code files under a directory, and we want to find which file contained the function that we wrote 2 months ago, then running the `grep -R function_name .` command can help us with it.

There is also another similar library called `ripgrep` that has better syntax highlighting, more flags...., for example:

`rg -u --files-without-match "^#/!" -t sh`, `-u` means don't ignore hidden files, and the command finds all the `sh` files that doesn't have a shebang.

### Finding commands
Ctrl+R: enters recall mode to search for commands that we have previous used, for example, if we want to find all the `grep` commands that we have used befor, then Ctrl+R, `grep` and keeping hitting the uparrow key until we find what we want. 

We can also use the `history` command, to start from the history of the first command, do `history 1`, and then we can further pipe this to perhaps `grep` to refine our search, such as `history 1 | grep grep`

### Exercise 1:
Read `man ls` and write an `ls` command that lists files in the following manner
- Includes all files, including hidden files
- Sizes are listed in human readable format (e.g. 454M instead of 454279954)
- Files are ordered by recency
- Output is colorized

In [48]:
%%bash
man ls | grep human -
man ls | grep color -
man ls | grep \\-t -
ls --color=always -a -c -l -t -h 
# to reverse sort, use -r
printf "\nfor a shorter command \n"
ls --color=always -alth

       -h, --human-readable
              with -l and/or -s, print human readable sizes (e.g., 1K 234M 2G)
       --color[=WHEN]
              colorize  the output; WHEN can be 'always' (default if omitted),
       -f     do not sort, enable -aU, disable -ls --color
       Using  color  to distinguish file types is disabled both by default and
       with --color=never.  With --color=auto, ls emits color codes only  when
       variable can change the settings.  Use the dircolors command to set it.
       --file-type
       --full-time
              like -l --time-style=full-iso
       --dereference-command-line-symlink-to-dir
              slash (-p), file-type (--file-type), classify (-F)
              sort  by  WORD instead of name: none (-U), size (-S), time (-t),
       --time=WORD
       --time-style=STYLE
       -t     sort by modification time, newest first
       -T, --tabsize=COLS
total 64K
drwxr-xr-x 5 lukezhu lukezhu 4.0K Aug 27 11:21 .
-rw-r--r-- 1 lukezhu lukezhu  11K Aug

### Exercise 2
Write bash functions `marco` and `polo` that do the following. Whenever you execute `marco` the current working directory should be saved in some manner, then when you execute `polo`, no matter what directory you are in, `polo` should `cd` you back to the directory where you executed `marco`. For ease of debugging you can write the code in a file `marco.sh` and (re)load the definitions to your shell by executing source `marco.sh`.

In [103]:
%%bash
# mkdir ./lecture2_exercises
cd lecture2_exercises
# << is known as here-document, the word following '<<' acts as the delimiter, and if there are quotes around the 
# word, then it prevents variables being evaluated before writing to file
cat > marco.sh << 'EOF' # make sure to add single quotes around EOF to prevent variable ($pwd) from evaluating
#!bin/bash
marco() {
    prev_dir=$(pwd)
}

# comparing 

polo() {
    cd $prev_dir
}
EOF

source ./marco.sh

marco
cd ..
echo $(pwd)
polo
echo $(pwd)

/home/lukezhu/Code/missing-semester
/home/lukezhu/Code/missing-semester/lecture2_exercises


## Exercise 3
Say you have a command that fails rarely. In order to debug it you need to capture its output but it can be time consuming to get a failure run. Write a bash script that runs the following script until it fails and captures its standard output and error streams to files and prints everything at the end. Bonus points if you can also report how many runs it took for the script to fail.
```
#!/usr/bin/env bash

 n=$(( RANDOM % 100 ))

 if [[ n -eq 42 ]]; then
    echo "Something went wrong"
    >&2 echo "The error was using magic numbers"
    exit 1
 fi

 echo "Everything went according to plan"
```

In [185]:
%%bash
pwd
cd lecture2_exercises
cat >exercise3 <<'EOF'
#!/usr/bin/env bash

n=$(( RANDOM % 100 ))

if [ $n -eq 42 ]; then
echo "Something went wrong"
>&2 echo "The error was using magic numbers"
exit 1
fi

echo "Everything went according to plan"
EOF


cat >run_exercise3 <<'EOF'
#!/usr/bin/env bash
i=0
execution_status=0
while [ $execution_status -eq 0 ]; do
    ./exercise3 >/dev/null
    execution_status=$?
    i=$((i+1))
done
echo "the total number of iterations were $i"
EOF

/home/lukezhu/Code/missing-semester


In [186]:
%%bash
cd lecture2_exercises
chmod +x exercise3 run_exercise3
./run_exercise3

the total number of iterations were 40


The error was using magic numbers


## Exercise 4
As we covered in the lecture `find`’s `-exec` can be very powerful for performing operations over the files we are searching for. However, what if we want to do something with all the files, like creating a zip file? As you have seen so far commands will take input from both arguments and `STDIN`. When piping commands, we are connecting `STDOUT` to `STDIN`, but some commands like `tar` take inputs from arguments. To bridge this disconnect there’s the xargs command which will execute a command using `STDIN` as arguments. For example `ls | xargs rm` will delete the files in the current directory.

Your task is to write a command that recursively finds all HTML files in the folder and makes a zip with them. Note that your command should work even if the files have spaces (hint: check -d flag for xargs)

In [191]:
%%bash
cd lecture2_exercises
for i in {1..3}
do
    touch "random$i.html"
    touch "random $i.html"
done

In [12]:
%%bash
cd lecture2_exercises
# need to pass in *.html inside '' to avoid shell expansion prior to executing find
# man xargs
find . -name '*.html' | xargs -0 -d '\n' tar -cvf file.tar.gz
# -d ' ' tar -cvf file.tar.gz \;

./random 1.html
./random1.html
./random2.html
./random 2.html
./random3.html
./random 3.html


### Exercise 5
(Advanced) Write a command or script to recursively find the most recently modified file in a directory. More generally, can you list all files by recency?

In [18]:
%%bash
man tail

TAIL(1)                          User Commands                         TAIL(1)

NAME
       tail - output the last part of files

SYNOPSIS
       tail [OPTION]... [FILE]...

DESCRIPTION
       Print  the  last  10  lines of each FILE to standard output.  With more
       than one FILE, precede each with a header giving the file name.

       With no FILE, or when FILE is -, read standard input.

       Mandatory arguments to long options are  mandatory  for  short  options
       too.

       -c, --bytes=[+]NUM
              output  the  last  NUM  bytes; or use -c +NUM to output starting
              with byte NUM of each file

       -f, --follow[={name|descriptor}]
              output appended data as the file grows;

              an absent option argument means 'descriptor'

       -F     same as --follow=name --retry

       -n, --lines=[+]NUM
              output the last NUM lines, instead of the last  10;  or  use  -n
              +NUM to output starting with line NUM

    

In [20]:
%%bash
ls -l -t | tail -n +2 | head -1
# or, reverse sort and output the last line
ls -ltr | tail -n 1 | xargs echo

-rw-r--r-- 1 lukezhu lukezhu 22537 Sep  3 19:40 Lecture 2.ipynb
-rw-r--r-- 1 lukezhu lukezhu 22537 Sep 3 19:40 Lecture 2.ipynb


In [71]:
%%bash
cd lecture2_exercises
cat >'exercise5' <<'EOF'
#!bin/bash
list_most_recent_file() {
    dir_name=$1
    ls $dir_name -lt | tail -n +2 | head -1
}
# list_most_recent_file()
#
EOF

In [72]:
%%bash
cd lecture2_exercises
source exercise5

list_most_recent_file $(pwd)

-rw-r--r-- 1 lukezhu lukezhu   127 Sep  3 20:11 exercise5
